In [21]:
#import databases
from datetime import datetime
import matplotlib.pyplot as plt
import netCDF4
from FireWeatherIndex import FWICLASS
import numpy as np
from tqdm import tqdm
from MatrixCalculations import MATRIXCALC

In [22]:
#loading files
folder = "Guido-data/"
temp = netCDF4.Dataset(folder + "tg_2024050112-14H.nc", "r")
rhum = netCDF4.Dataset(folder + "hu_2024050112-14H.nc", "r")
wind = netCDF4.Dataset(folder + "fg_2024050112-14H.nc", "r")
prcp = netCDF4.Dataset(folder + "rr_2024050112-dummy.nc", "r")

In [23]:
#Finding dimensions of one of the files: The files are of the same size, here they are found of the air_temperature. The starting files for
#the different starting variables have to be of the same size to start the algorithm.
latitude = temp.variables['air_temperature'].shape[1]
longitude = temp.variables['air_temperature'].shape[2]

In [24]:
#writing files in a netCDF4 file
ffmcfile = netCDF4.Dataset('ffmcfile.nc', mode='w', format='NETCDF4_CLASSIC')

lat_dim = ffmcfile.createDimension('lat', latitude)  # latitude axis
lon_dim = ffmcfile.createDimension('lon', longitude)  # longitude axis
time_dim = ffmcfile.createDimension('time', None)  # unlimited axis (can be appended to).

In [25]:
#Attributes
ffmcfile.title = 'ffmc_test'

In [26]:
#Creating variables
lat = ffmcfile.createVariable('lat', np.float32, ('lat',))
lat.units = 'degrees_north'
lat.long_name = 'latitude'
lon = ffmcfile.createVariable('lon', np.float32, ('lon',))
lon.units = 'degrees_east'
lon.long_name = 'longitude'
time = ffmcfile.createVariable('time', np.float64, ('time',))
time.units = 'days from 1-5-2024'
time.long_name = 'time'
ffmc_index = ffmcfile.createVariable('ffmc', np.float64,
                                     ('time', 'lat', 'lon'))  # note: unlimited dimension is leftmost
ffmc_index.units = '1'
isi_index = ffmcfile.createVariable('isi', np.float64, ('time', 'lat', 'lon'))  # note: unlimited dimension is leftmost
isi_index.units = '1'

In [27]:
#Here we create the FFMC0 matrix, so we started with 85, and from here we calculate further. The base value is also set on 85
#It's easily changed to another value. The matrices will converge to a close value after about 3 days.
ffmc0=85.0

value = MATRIXCALC(ffmc0, longitude, latitude)
begin = value.MAKE_MAT_FFMC0()

ffmc_index[0, :, :] = begin.transpose()

isi_index[0, :, :] = begin

In [28]:
i=0
while i<11:
    FFMC=ffmc_index[i, :, :]
    temperature=temp['air_temperature'][i, :, :].transpose()
    rhumidity=rhum['relative_humidity'][i, :, :].transpose()
    windspeed=wind['windspeed'][0, :, :].transpose()
    new_slice = value.MAKE_NEXT_MATRIX_FFMC(FFMC=FFMC.transpose(), temp=temperature, rhum=rhumidity, wind=windspeed)
    i+=1
    ffmc_index[i, :, :] = new_slice.transpose()


The precipitation are set to 0 everywhere, only use this as a test.

The precipitation are set to 0 everywhere, only use this as a test.

The precipitation are set to 0 everywhere, only use this as a test.

The precipitation are set to 0 everywhere, only use this as a test.

The precipitation are set to 0 everywhere, only use this as a test.

The precipitation are set to 0 everywhere, only use this as a test.

The precipitation are set to 0 everywhere, only use this as a test.

The precipitation are set to 0 everywhere, only use this as a test.

The precipitation are set to 0 everywhere, only use this as a test.

The precipitation are set to 0 everywhere, only use this as a test.

The precipitation are set to 0 everywhere, only use this as a test.


In [29]:
ffmcfile.close(); print('Dataset is closed')

Dataset is closed
